In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [52]:
# Load dataset

PlayerStats = pd.read_csv('player_data.csv')
PlayerStats

,Rk,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,year
0,1,Nicholas Abruzzese,23.0,TOR,C,2,0,2,2,1,...,2,0.0,18,8:59,0.0,1,0.0,0.0,NaN,2023
1,2,Noel Acciari,31.0,TOT,C,77,14,9,23,-4,...,102,13.7,1102,14:19,81.0,244,337.0,289.0,53.8,2023
2,2,Noel Acciari,31.0,STL,C,54,10,8,18,-6,...,74,13.5,787,14:35,55.0,168,275.0,239.0,53.5,2023
3,2,Noel Acciari,31.0,TOR,C,23,4,1,5,2,...,28,14.3,315,13:41,26.0,76,62.0,50.0,55.4,2023
4,3,Calen Addison,22.0,MIN,D,62,3,26,29,-17,...,77,3.9,999,16:07,38.0,19,0.0,0.0,NaN,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14627,875,Greg Zanon,29.0,MIN,D,81,2,13,15,-10,...,59,3.4,1812,22:22,196.0,183,0.0,1.0,0.0,2010
14628,876,Henrik Zetterberg,29.0,DET,C,74,23,47,70,12,...,309,7.4,1485,20:04,34.0,44,536.0,543.0,49.7,2010
14629,877,Vladimir Zharkov,22.0,NJD,RW,40,0,10,10,2,...,54,0.0,458,11:26,6.0,22,1.0,3.0,25.0,2010
14630,878,Marek Židlický,32.0,MIN,D,78,6,37,43,-16,...,116,5.2,1885,24:10,128.0,80,0.0,0.0,NaN,2010


In [8]:
PlayerStats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14632 entries, 0 to 14631
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      14632 non-null  int64  
 1   Player  14632 non-null  object 
 2   Age     14630 non-null  float64
 3   Tm      14632 non-null  object 
 4   Pos     14632 non-null  object 
 5   GP      14632 non-null  int64  
 6   G       14632 non-null  int64  
 7   A       14632 non-null  int64  
 8   PTS     14632 non-null  int64  
 9   +/-     14632 non-null  int64  
 10  PIM     14632 non-null  int64  
 11  PS      14632 non-null  float64
 12  EV      14632 non-null  int64  
 13  PP      14632 non-null  int64  
 14  SH      14632 non-null  int64  
 15  GW      14632 non-null  int64  
 16  EV.1    14632 non-null  int64  
 17  PP.1    14632 non-null  int64  
 18  SH.1    14632 non-null  int64  
 19  S       14632 non-null  int64  
 20  S%      14243 non-null  float64
 21  TOI     14632 non-null  int64  
 22

In [53]:
PlayerStats.groupby('Player')['year'].size().sort_values(ascending=False)

Player
Derick Brassard          23
Nate Thompson            21
Luke Schenn              20
Ian Cole                 19
Vladislav Namestnikov    19
                         ..
Sean Day                  1
Sean Farrell              1
Jamie Doornbosch          1
Matt Anderson             1
Carter Bancks             1
Name: year, Length: 2602, dtype: int64

We only want to keep stats for total year, so when we have cases where a player changes team during the year, we only want to keep the TOT statline

In [54]:
# intialize empty list to store rows
rows_to_keep = []

# Create set to keep track of 'Player' values for each year
encountered_players = {}

# Iterate through dataframe
for index, row in PlayerStats.iterrows():
    player = row['Player']
    team = row['Tm']
    year = row['year']

    # Check if player has been encountered before
    if year not in encountered_players:
        encountered_players[year] = set()
    if player not in encountered_players[year]:
        # If first instance encountering player in year, add row
        rows_to_keep.append(row)
        encountered_players[year].add(player)
    elif team == "TOT":
        # if has 'TOT' value in Tm, insert this value
        for i, stored_row in enumerate(rows_to_keep):
            if stored_row['Player'] == player and stored_row['year'] == year:
                rows_to_keep[i] = row

PlayerStats2 = pd.DataFrame(rows_to_keep)
PlayerStats2.reset_index(inplace = True, drop = True)

PlayerStats2

,Rk,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,year
0,1,Nicholas Abruzzese,23.0,TOR,C,2,0,2,2,1,...,2,0.0,18,8:59,0.0,1,0.0,0.0,NaN,2023
1,2,Noel Acciari,31.0,TOT,C,77,14,9,23,-4,...,102,13.7,1102,14:19,81.0,244,337.0,289.0,53.8,2023
2,3,Calen Addison,22.0,MIN,D,62,3,26,29,-17,...,77,3.9,999,16:07,38.0,19,0.0,0.0,NaN,2023
3,4,Egor Afanasyev,22.0,NSH,LW,17,1,0,1,-7,...,21,4.8,184,10:51,13.0,23,0.0,0.0,NaN,2023
4,5,Andrew Agozzino,32.0,SJS,LW,4,1,2,3,-1,...,9,11.1,65,16:22,4.0,12,1.0,0.0,100.0,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12594,875,Greg Zanon,29.0,MIN,D,81,2,13,15,-10,...,59,3.4,1812,22:22,196.0,183,0.0,1.0,0.0,2010
12595,876,Henrik Zetterberg,29.0,DET,C,74,23,47,70,12,...,309,7.4,1485,20:04,34.0,44,536.0,543.0,49.7,2010
12596,877,Vladimir Zharkov,22.0,NJD,RW,40,0,10,10,2,...,54,0.0,458,11:26,6.0,22,1.0,3.0,25.0,2010
12597,878,Marek Židlický,32.0,MIN,D,78,6,37,43,-16,...,116,5.2,1885,24:10,128.0,80,0.0,0.0,NaN,2010


In [49]:
PlayerStats2['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010])

In [55]:
PlayerStats2.groupby('Player')['year'].size().sort_values(ascending=False)

Player
Erik Johnson              14
Kris Letang               14
Patrice Bergeron          14
Patric Hörnqvist          14
Wayne Simmonds            14
                          ..
Glen Metropolit            1
Ole-Kristian Tollefsen     1
Grant Hutton               1
Callahan Burke             1
Joonas Nättinen            1
Name: year, Length: 2602, dtype: int64

In [57]:
PlayerStats['Player'].nunique()

2602

In [58]:
PlayerStats2['Player'].nunique()

2602